# Coursera assignment - Applied Data Science - Week 3

## Importing required libraries

In [156]:
import pandas as pd
import numpy as np
#import requests
#import urllib
#import time
#import json

## Pulling the table from Wiki Link

In [157]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[0]
df = df[1:]
df.columns = ["Postcode", "Borough", "Neighbourhood"]

## Cleaning up the missing data and concatenating duplicate neighbourhood

In [158]:
df = df[df['Borough']!="Not assigned"]

In [159]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighbourhood'].tolist()))
                    
df_grp = df.groupby(['Postcode', 'Borough'])
df2 = df_grp.apply(neighborhood_list).reset_index(name='Neighbourhood')

In [160]:
df2['Neighbourhood'] = np.where(df2['Neighbourhood']=="Not assigned",df2['Borough'] ,df2['Neighbourhood'])

## Checking the number of rows and colums in the data and sample data

In [161]:
df2.shape

(103, 3)

In [162]:
df2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Importing geocoder and defining function to pull lat long data

In [163]:
import geocoder

def pull_latlng(Postcode):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(Postcode))
        lat_lng_coords = g.latlng
    return lat_lng_coords

## Pull data for all the post code using the above UDF and for loop

In [164]:
pl_data = [pull_latlng(Postcode) for Postcode in df2['Postcode'].tolist()]

## cleaning the data and merging it with the original dataframe

In [165]:
df_pl_data = pd.DataFrame(pl_data, columns=['Latitude', 'Longitude'])
df2['Latitude'] = df_pl_data['Latitude']
df2['Longitude'] = df_pl_data['Longitude']

## Checking the number of rows and colums in the data and sample data

In [166]:
df2.shape

(103, 5)

In [168]:
df2.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
